# Main script

In [ ]:
import itertools
import json
import os
import warnings
import zipfile

from joblib import Parallel, delayed, parallel_backend
from joblib.externals.loky import get_reusable_executor
import numpy as np
from numpy import arange
from tqdm.notebook import tqdm

In [2]:
from pNeuma_simulator.results import zipdir
from pNeuma_simulator.simulate import batch
from pNeuma_simulator import params

In [3]:
warnings.filterwarnings("ignore")

In [4]:
# Joblib nested Parallel execution: toy example
# https://stackoverflow.com/questions/67891651/
scale = 10
reps = 8
n_jobs = 16
n_threads = 4
# Caution: effective n_jobs==reps !!!


def parallel_in_parallel_test(i):
    a = np.ones((scale, scale))
    for j in range(5000):
        a *= np.random.randn(scale, scale)
    return a.sum()


def parallel_in_parallel_wrapper(j, n_threads=4):
    with parallel_backend("loky", inner_max_num_threads=n_threads):
        with Parallel(n_jobs=n_threads) as parallel:
            out2 = parallel(delayed(parallel_in_parallel_test)(i) for i in range(100))
    return np.array(out2).sum()


with Parallel(n_jobs=n_jobs) as parallel:
    out = parallel(delayed(parallel_in_parallel_wrapper)(j, n_threads=n_threads) for j in range(reps))
# https://stackoverflow.com/questions/67495271/
get_reusable_executor().shutdown(wait=True)

In [5]:
path = "./output/"
os.makedirs(path, exist_ok=True)
# Combinatorial configurations
n_veh = 8
scale = 2
l_cars = scale * arange(1, n_veh, 1)
l_moto = scale * arange(0, n_veh - 1, 1)
permutations = list(itertools.product(l_cars, l_moto))
len(permutations)

49

In [6]:
epochs = 64
default_rng = np.random.default_rng(1024)
seeds = default_rng.integers(1e8, size=epochs * len(permutations))
name = (12, 6)
n_jobs = 64
n_threads = 1
save = True

for n, permutation in tqdm(enumerate(permutations)):
    start = n * epochs
    end = (n + 1) * epochs
    if permutation == name:
        # https://stackoverflow.com/questions/67891651/
        with Parallel(n_jobs=n_jobs) as parallel:
            items = parallel(delayed(batch)(seed, permutation, n_jobs=n_threads) for seed in seeds[start:end])
        # https://stackoverflow.com/questions/67495271/
        get_reusable_executor().shutdown(wait=True)
        if save:
            # Dump to JSONL https://jsonlines.org/examples/
            # https://stackoverflow.com/questions/38915183/
            with open(f"{path}{permutation}.jsonl", "w") as outfile:
                for item in items:
                    json.dump(item, outfile)
                    outfile.write("\n")
            # Compress as Zip and delete original JSONL
            with zipfile.ZipFile(f"{path}{permutation}_.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
                zipdir(path, zipf)
        print(permutation)

0it [00:00, ?it/s]

(12, 6)
